In [1]:
import numpy as np

from weight import *
from utils import save, load
from library import *
from process_library_terms import *

In [22]:
%%prun # profiling

### load data
# vicsek:
#arrays.npy - normal. arrays2.npy - very few particles. arrays3.npy - high density. arrays4.npy - high density, low noise.
#arrays5.npy - small domain, one clump
#datasets = ['arrays.npy', 'arrays2.npy', 'arrays3.npy']#, 'arrays5.npy']
#datasets = ['data_bu.npy'] # burgers' equation data
## it may be interesting to reintroduce regression on multiple datasets,
## but for now we're just not going to do it

#dataset = 'data_bu.npy'
#dataset = 'lamp2.npy'
dataset = 'lamp2_1e-4.npy'
#import scipy.io
#dataset = 'gas_1000_4_0.mat'
#dataset = 'gas_1000_16_4.mat'
#dataset = 'gas_5000_24_4.mat'
#matdata = scipy.io.loadmat(dataset)
#positions = matdata['pos_data']
#vs = matdata['vs_data']
#deltat = matdata['dt'][0][0]
#dims = matdata['dims'][0]
#for i, dset in enumerate(datasets):
positions, vs, deltat, dims = load(dataset, 4)
deltat = float(deltat)
world_size = np.array([dims[0], dims[0], dims[1]])
print(world_size)
cg_res = 200#20
#world_size[2] = world_size[2]-200 # start at frame 200
#print(positions.shape)
#positions = positions[:, :, 200:]
#vs = vs[:, :, 200:]
#world_size[2] = 50 # restrict to first 50 time steps
data_dict = {}
data_dict['v'] = vs#*2 # because scaling could be wrong
v_obs = Observable('v', 1)
observables = [v_obs]

# fix random seed
np.random.seed(1)

# initial setup of dataset
srd = SRDataset(world_size, data_dict, positions, observables, 
                kernel_sigma=0.04, cg_res=cg_res, deltat=deltat, cutoff=6)
#kernel_sigma=0.05 (lamp),0.1 (Jason?),0.6 (burgers?)

# initialize libraries, domains, and weights
#srd.make_libraries(max_complexity=4, max_observables=3)
srd.make_libraries(max_complexity=5, max_observables=4)
#srd.make_libraries(max_complexity=6, max_observables=4)

dom_width = 0.2#1.5 #10 #80
dom_time = 20 #50
srd.make_domains(ndomains=30, domain_size=[dom_width, dom_width, dom_time], pad=5)
#srd.make_domains(ndomains=1, domain_size=[dom_width, dom_width, dom_time], pad=0)
srd.make_weights(m=16, qmax=1)
srd.make_library_matrices(debug=False)

save('Q_bu.npy', srd.dxs, srd.libs)

[   1.01138514    1.01138514 1001.        ]
 

         83122087 function calls (82020496 primitive calls) in 810.758 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      390  173.683    0.445  699.352    1.793 process_library_terms.py:187(eval_cgp)
  1712360  143.416    0.000  357.465    0.000 convolution.py:15(gauss1d)
  3748980   68.165    0.000   68.165    0.000 {method 'reduce' of 'numpy.ufunc' objects}
  2072384   44.574    0.000   44.574    0.000 {built-in method numpy.array}
  1712360   42.312    0.000   42.312    0.000 {method 'outer' of 'numpy.ufunc' objects}
  1768999   41.599    0.000   41.599    0.000 {built-in method builtins.sum}
  3542092   32.347    0.000  116.504    0.000 fromnumeric.py:69(_wrapreduction)
  1712360   25.034    0.000   61.527    0.000 shape_base.py:512(expand_dims)
6015072/5544589   24.439    0.000  248.244    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
   108720   21.787    0.000   22.328    0.000 pol

In [23]:
_, libs = load('Q_bu.npy', 2)
libs = libs.item()

In [26]:
from identify_models import *
opts = dict()
opts['threshold'] = "'verbose'"
opts['row_norms'] = libs[0].row_weights
opts['gamma'] = 2
opts['verbose'] = False
opts['delta'] = 1e-8#1e-8 # apparently this reaches machine precision as the relative error
opts['char_sizes'] = libs[0].col_weights
#opts['verbose'] = True
opts1 = opts.copy()
opts1['char_sizes'] = libs[1].col_weights
opts1['row_norms'] = libs[1].row_weights

Q = libs[0].Q
#print(Q[:, 3]+Q[:, 14], np.linalg.norm(Q[:, 3]+Q[:, 14]), 
#np.linalg.norm(Q[:, 3]+Q[:, 14])/np.linalg.norm(Q[:, 0]))
#print(np.linalg.norm(Q[:, 0])/libs[0].col_weights[0])
#print(list(zip(libs[1].terms, libs[1].col_weights)))
#eqs0, lambdas0, derived_eqs0, excluded_terms = identify_equations(Q0, opts, terms0, observables, threshold=1e-5, 
#                       max_complexity=None, max_equations=999, timed=True)
#print(excluded_terms)
#eqs1, lambdas1, derived_eqs1, excluded_terms = identify_equations(Q1, opts1, terms1, observables, threshold=1e-5, 
#                       max_complexity=None, max_equations=999, timed=True, excluded_terms=excluded_terms)
#print(excluded_terms)
eqs, lambdas, derived_eqs, excluded_terms = interleave_identify([libs[0].Q, libs[1].Q], 
[opts, opts1], [libs[0].terms, libs[1].terms], threshold=1e-5)


#[0.15 s]
#Identified model: -0.9787526520510974 * rho + 0.028230207633854636 * rho * rho + -0.028610932601784253 * rho * rho[v_j * v_j] + 1.0 * rho[v_j * v_j] = 0 (order 5, residual 5.07e-05)
#[0.15 s]
#Identified model: -0.02837528778491848 * rho * rho + 1.0 * rho[v_j * v_j] = 0 (order 6, residual 1.43e-05)
#[0.26 s]
#Identified model: 0.9999199152531711 * dj rho[v_j] + 1.0 * dt rho = 0 (order 6, residual 3.51e-06)

[0.64 s]
Identified model: -0.0015634362723435191 * rho * rho[v_i] + 1.0 * rho[v_i] = 0 (order 4, residual 9.58e-06)
[3.76 s]
Identified model: -0.9313287472640646 * rho[v_i] * rho[v_j * v_j] + 1.0 * rho[v_j] * rho[v_i * v_j] = 0 (order 5, residual 6.14e-09)
[7.27 s]
Identified model: -8.673676428823279e-05 * dt rho[v_i * v_j * v_j] + 1.0 * rho[v_j] * rho[v_i * v_j] = 0 (order 5, residual 1.55e-08)
[10.29 s]
Identified model: -0.12187484554802103 * rho[v_i * v_j * v_j] + 1.0 * rho[v_j] * rho[v_i * v_j] = 0 (order 5, residual 1.54e-08)
[12.94 s]
Identified model: -0.003546877914516303 * dt dj rho[v_i * v_j] + 1.0 * rho[v_i * v_j * v_j] = 0 (order 5, residual 7.42e-06)


In [6]:
print(len(libs[0].terms))
print(len(libs[1].terms))

98
109


In [17]:
print(np.std(vs))

0.5499157424054096


In [6]:
## mostly not planning to use this code

from ipynb.fs.full.sparse_reg import *
opts = dict()
opts['threshold'] = "'pareto'"
#opts['verbose'] = True
#opts['brute_force'] = False
#opts['threshold'] = "'threshold'"
#opts['n_terms'] = len(terms0)
#opts['gamma'] = 1.2
opts['char_sizes'] = srd.libs[0].col_weights
Xi, lambd, best_term, lambda1 = sparse_reg(srd.libs[0].Q, opts=opts)
print(srd.libs[0].Q.shape)

(240, 28)


In [8]:
for i in range(len(Xi)):
    if Xi[i]!=0:
        print(f"[Term {i}] {Xi[i]} * {srd.libs[0].terms[i]}")
print("Model residual:", lambd)
print("One-term model residual:", lambda1)
print(f"Best term (#{best_term}):", srd.libs[0].terms[best_term])

[Term 19] 1.0 * rho[v_j] * rho[v_j]
Model residual: 0.0030944418121483697
One-term model residual: 1.294046607514735e-05
Best term (#6): dt^3 rho


In [ ]:
opts['verbose'] = True
opts['char_sizes'] = srd.libs[1].col_weights
opts['avoid'] = []
opts['row_norms'] = None
Xi, lambd, best_term, lambda1 = sparse_reg(srd.libs[1].Q, opts=opts)
print(srd.libs[1].Q.shape)

In [ ]:
for i in range(len(Xi)):
    if Xi[i]!=0:
        print(f"[Term {i}] {Xi[i]} * {srd.libs[1].terms[i]}")
print("Model residual:", lambd)
print("One-term model residual:", lambda1)
print("Best term:", srd.libs[1].terms[best_term])

In [24]:
for term, size in zip(libs[0].terms, libs[0].col_weights):
    print(term, size)

1 1
rho 25.330295910584443
dj^2 rho 0.4862908401198736
dj^2 dk^2 rho 0.001215727100299684
dt rho 0.9725816802397471
dt dj^2 rho 0.0024314542005993683
dt^2 rho 0.004862908401198736
dt^2 dj^2 rho 1.215727100299684e-05
dt^3 rho 2.431454200599368e-05
dt^4 rho 1.2157271002996838e-07
rho * rho 641.623890917771
rho * dj^2 rho 12.317890878843107
rho * dt rho 24.63578175768621
rho * dt dj^2 rho 0.06158945439421554
rho * dt^2 rho 0.12317890878843105
rho * dt^3 rho 0.0006158945439421553
rho * rho * rho 16252.523020247692
rho * rho * dj^2 rho 312.015820955385
rho * rho * dt rho 624.0316419107698
rho * rho * dt^2 rho 3.1201582095538494
rho * rho * rho * rho 411681.21739645966
dj rho[v_j] 6.3772721379084665
dj^2 dk rho[v_k] 0.01594318034477117
dt dj rho[v_j] 0.031886360689542335
dt dj^2 dk rho[v_k] 7.971590172385585e-05
dt^2 dj rho[v_j] 0.00015943180344771168
dt^3 dj rho[v_j] 7.971590172385584e-07
rho * dj rho[v_j] 161.53819035554693
rho * dt dj rho[v_j] 0.8076909517777346
rho * rho * dj rho[v_j] 40

In [27]:
continuity_terms = ['dt rho', 'dj rho[v_j]']
#continuity_terms = ['rho * dt rho', 'rho * dj rho[v_j]']
#continuity_terms = ['dt rho * rho', 'rho * rho * dj v_j', 'dj rho * rho * v_j']
#continuity_terms = ['dt rho', 'dj rv_j']
col_numbers = [find_term(libs[0].terms, name) for name in continuity_terms]
Xi, lambd = regress(libs[0].Q, col_numbers)
for i in range(len(Xi)):
    if Xi[i]!=0:
        print(f"[Term {i}] {Xi[i]} * {libs[0].terms[i]}.")# (Char. size: {srd.libs[0].col_weights[i]})")
print("Model residual:", lambd)

[Term 3] 1.0 * dt rho.
[Term 13] 0.7385727675285971 * dj rho[v_j].
Model residual: 49.54248695349633


In [28]:
from scipy.stats.stats import pearsonr 
print(np.vstack([libs[0].Q[1:10, 3],0.7237*libs[0].Q[1:10, 13]]))
print(libs[0].Q[1:10, 3]+0.7237*libs[0].Q[1:10, 13])
print(pearsonr(libs[0].Q[1:10, 3], libs[0].Q[1:10, 13]))

[[-17213.17856145   1918.77457133 -15725.91116491 -12633.29306777
  -22781.89587622   1975.39933656   4679.5848596    7117.28893318
  -21033.10662767]
 [ 15292.05883415  -2860.15527405  14125.54192429  12047.77069629
   22900.21679984   -136.19714598  -5552.37519339  -5588.83755774
   23336.02250945]]
[-1921.1197273   -941.38070271 -1600.36924063  -585.52237148
   118.32092363  1839.20219057  -872.79033379  1528.45137543
  2302.91588177]
(-0.9915217242456649, 1.8324333539848543e-07)


In [27]:
burger_terms = ['dt rho[v_i]', 'dj rho[v_i * v_j]']
#burger_terms = ['rho[v_i]', 'rho[v_i * v_j * v_j]']
#burger_terms = ['rho * dt rv_i', 'dj rv_i * rv_j']
col_numbers = [find_term(libs[1].terms, name) for name in burger_terms]
Xi, lambd = regress(libs[1].Q, col_numbers)
for i in range(len(Xi)):
    if Xi[i]!=0:
        print(f"[Term {i}] {Xi[i]} * {libs[1].terms[i]}. (Char. size: {libs[1].col_weights[i]})")
print("Model residual:", lambd)

[Term 10] 1.0 * dt rho[v_i]. (Char. size: 677.6742656944556)
[Term 19] 0.0003867473227255034 * dj rho[v_i * v_j]. (Char. size: 18.633187346420367)
Model residual: 0.009782908807667946


In [14]:
### not fixed (obviously)
#veqconst_terms = ['1', 'v_j * v_j']
veqconst_terms = ['rho', 'rho * v_j * v_j']
col_numbers = [find_term(terms0, name) for name in veqconst_terms]
Xi, lambd = regress(Q0, col_numbers)
for i in range(len(Xi)):
    if Xi[i]!=0:
        print(f"[Term {i}] {Xi[i]} * {terms0[i]}. (Char. size: {char_sizes0[i]})")
print("Model residual:", lambd)

[Term 16] -0.004286563115825593 * rho. (Char. size: 0.1464261231728444)
[Term 28] 1.0 * rho * v_j * v_j. (Char. size: 0.0005121399208783936)
Model residual: 0.0005697337514007149


In [ ]:
rho dt^2 v = 10^-4 dt rho * grad rho